In [1]:
"""
MIT License

Copyright (c) 2017 Ryan Whytsell

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

# Modified by Harish Iyer Ganesan

'\nMIT License\n\nCopyright (c) 2017 Ryan Whytsell\n\nPermission is hereby granted, free of charge, to any person obtaining a copy\nof this software and associated documentation files (the "Software"), to deal\nin the Software without restriction, including without limitation the rights\nto use, copy, modify, merge, publish, distribute, sublicense, and/or sell\ncopies of the Software, and to permit persons to whom the Software is\nfurnished to do so, subject to the following conditions:\n\nThe above copyright notice and this permission notice shall be included in all\ncopies or substantial portions of the Software.\n\nTHE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\nIMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\nFITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\nAUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\nLIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FRO

In [2]:
import sys
import pygame
import random
from threading import Thread
import time
#import PreyPredator

import warnings
warnings.filterwarnings("ignore")

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import random
import sys
import copy
random.seed(None)

import csv

In [4]:
class Node:

    species = None
    x = None
    y = None
    health = None

    def __init__(self, spec=None):
        if spec is not None:
            self.species = spec
        else:
            self.species = 0
        self.x = None
        self.y = None
        self.health = self.species*2

    def set_location(self, pos_x, pos_y):
        self.x = pos_x
        self.y = pos_y

    def set_species(self, num):
        if num == 0 or num == 1 or num == 2:
            self.species = num
            if num is 0:  # Blank
                self.health = None
            if num is 1:  # Predator
                self.health = 2
            if num is 2:  # Prey
                self.health = 4
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def move_here(self, species, health):
        if species == 0 or species == 1 or species == 2:
            self.species = species
            if species is 0:  # Blank
                self.health = health
            if species is 1:  # Predator
                self.health = health
            if species is 2:  # Prey
                self.health = health
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def prey_eat(self):
            self.species = 1

    def prey_reproduce(self):
        self.species = 2
        self.health = 1

In [5]:
class Map:
    height = 100
    width = 100
    play_board = []
    prey_kill_count = 0

    def __init__(self):
        prey = 0
        predator = 0
        empty = 0
        for x in range(self.width):
            row = []
            for y in range(self.height):
                i = random.randint(0, 100)
                if i <= 40:
                    row.append(Node(2))
                    prey += 1
                elif i <= 80:
                    row.append(Node(1))
                    predator += 1
                else:
                    row.append(Node(0))
                    empty += 1

            self.play_board.append(row)
        print('Map created')
        print('Prey: ', prey)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('Width: ', len(self.play_board))
        print('Height of row 5: ', len(self.play_board[5]))
        
    def map_count(self, time_step,writer):
        pb = self.play_board
        prey = 0
        predator = 0
        empty = 0
        for x in range(0, self.width):
            for y in range(0, self.height):
                if  pb[x][y].species is 2:
                    prey += 1
                elif pb[x][y].species is 1:
                    predator += 1
                else:
                    empty += 1
                
                
        print('Time Step: ',time_step)
        print('Prey: ', prey)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('-----------------------------------------')
        writer.writerow({'time_step': time_step, 'prey': prey, 'predator': predator, 'empty': empty  })


    def get_board(self):
        return self.play_board

    def turn(self):
        pb = self.play_board
        [[self.check_neighbors(x, y, pb[x][y]) for y in range(0, self.height)] for x in range(0, self.width)]

    # Returns the neighbors of a node, If node is on edge this wraps
    def get_neighbors(self, x, y):
        top = self.play_board[x][(y - 1) % self.height]
        tr = self.play_board[(x + 1) % self.width][(y - 1) % self.height]
        right = self.play_board[(x + 1) % self.width][y]
        br = self.play_board[(x + 1) % self.width][(y + 1) % self.height]
        bottom = self.play_board[x][(y + 1) % self.height]
        bl = self.play_board[(x - 1) % self.width][(y + 1) % self.height]
        left = self.play_board[(x - 1) % self.width][y]
        tl = self.play_board[(x - 1) % self.width][(y - 1) % self.height]

        neighbors = [top, tr, right, br, bottom, bl, left, tl]
        return neighbors

    #
    # PREDATOR AND PREY CELLULAR AUTOMATON
    # The world is grid of cells, with 3 possibilities: Predator(Red, 1), Prey(Green, 2), or Empty(Black, 0).
    # Both predator and prey have a set health, that changes over time.
    # The simulation works in steps, with the following rules:
    #    -For prey:
    #        -Tries to move in a random direction.
    #        -Health increases.
    #        -When health reaches a threshold:
    #           -They will reproduce, creating a new "Prey"
    #            -Their health resets to 1
    #    -For predator:
    #        -Tries to move in a random direction.
    #        -Health decreases.
    #        -When health reaches 0, they die and turn into "Nothing".
    #       -If the adjacent square is a prey:
    #            -They will eat it, turning it into a "predator" (reproducing)
    #            -Their health will increase by the amount of health the eaten prey had
    def check_neighbors(self, x, y, node):
        # If the node is empty no checks are performed
        if node.species is 0:
            return

        if node.species is 1:
            # This is a predator. Check for prey if no prey reduce health/die
            neighbors = self.get_neighbors(x, y)
            prey = []
            open_ = []

            for neigh in neighbors:
                if neigh.species is 2:
                    prey.append(neigh)
                if neigh.species is 0:
                    open_.append(neigh)
            open_length = len(open_)

            if len(prey) is 0:
                if node.health > 0 and open_length > 0:
                    node.health -= 1
                    open_[random.randint(0, (open_length-1))].move_here(1, node.health)
                    node.set_species(0)
                else:
                    node.health -= 1
                    

            else:
                target = prey[random.randint(0, (len(prey)-1))]
                if node.health >= target.health:
                    target.prey_eat()
                    self.prey_kill_count+=1

            if node.health is None or node.health <= 0:
                node.species = 0
                return

        if node.species is 2:
            # Check for overpopulation/health This is prey
            neighbors = self.get_neighbors(x, y)
            open_spots = []
            for neigh in neighbors:
                if neigh.species is 0:
                    open_spots.append(neigh)
            count = len(open_spots)
            if count >= 7 or count < 1:
                node.health -= 2
            if node.health is None or node.health is 0:
                node.species = 0
                return
            if node.health > 6 and len(open_spots) >= 1:
                open_spots[random.randint(0, (len(open_spots))-1)].prey_reproduce()
                node.health = 4
                return
            node.health += 1

In [6]:
random.seed(None)
size = width, height = 500, 500
play_width = width/5
play_height = height/5
is_stopped = False

In [7]:
# These are the codes that pygame uses, I am not sure what format it is
black = 0
red = 16386570
blue = 658170
color_arr = [black, red, blue]

In [8]:
pygame.init()
screen = pygame.display.set_mode(size)
pp = Map()

Map created
Prey:  4075
Predator:  4003
Empty:  1922
Width:  100
Height of row 5:  100


In [9]:
# Paints a 3x3 square for each node
def paint(x, y, color_int):
    screen.set_at((x, y), color_arr[color_int])
    screen.set_at((x + 1, y), color_arr[color_int])
    screen.set_at((x, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 1), color_arr[color_int])
    screen.set_at((x + 2, y), color_arr[color_int])
    screen.set_at((x, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 2), color_arr[color_int])

In [10]:
# Creates random colors TESTING ONLY
def random_colors(w, h):
    ra = random.randint
    pa = paint
    while not is_stopped:
        for x in range(5, w, 5):
            for y in range(5, h, 5):
                color_int = ra(1, 2)
                pa(x, y, color_int)

In [11]:
# Paints the species at each point on the play board
def paint_map(board):
    pa = paint
    length = len(board)
    for x in range(0, length):
        for y in range(0, length):
            pa(x*5, y*5, board[x][y].species)

In [12]:
#### Game loop, paints the map then calculates the next turn
time_step=0
#open the file in the write mode
with open('1prey_1predator.csv', 'w', newline='') as csvfile:
    fieldnames = ['time_step', 'prey','intermediate_predator','predator','empty']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                is_stopped = True
                sys.exit(0)
        paint_map(pp.get_board())
        pygame.display.flip()
        pp.map_count(time_step, writer)
        pp.turn()
#         if(time_step==0 or time_step==200 or time_step==500 or time_step==1000):
#             a=input("Stop for snapshot")
        if(time_step==0 or time_step == 500):
            time.sleep(10)
            print("Prey Kill Count: ",pp.prey_kill_count)
            pygame.QUIT
            #a=input("Stop for snapshot")
        time_step+=1
            

Time Step:  0
Prey:  4075
Predator:  4003
Empty:  1922
-----------------------------------------
Prey Kill Count:  0
Time Step:  1
Prey:  4075
Predator:  4001
Empty:  1924
-----------------------------------------
Time Step:  2
Prey:  3753
Predator:  4312
Empty:  1935
-----------------------------------------
Time Step:  3
Prey:  3377
Predator:  4578
Empty:  2045
-----------------------------------------
Time Step:  4
Prey:  4373
Predator:  5296
Empty:  331
-----------------------------------------
Time Step:  5
Prey:  3714
Predator:  5275
Empty:  1011
-----------------------------------------
Time Step:  6
Prey:  3543
Predator:  5622
Empty:  835
-----------------------------------------
Time Step:  7
Prey:  3030
Predator:  5917
Empty:  1053
-----------------------------------------
Time Step:  8
Prey:  2726
Predator:  6387
Empty:  887
-----------------------------------------
Time Step:  9
Prey:  2192
Predator:  6354
Empty:  1454
-----------------------------------------
Time Step:  1

Time Step:  87
Prey:  253
Predator:  82
Empty:  9665
-----------------------------------------
Time Step:  88
Prey:  264
Predator:  85
Empty:  9651
-----------------------------------------
Time Step:  89
Prey:  268
Predator:  86
Empty:  9646
-----------------------------------------
Time Step:  90
Prey:  288
Predator:  85
Empty:  9627
-----------------------------------------
Time Step:  91
Prey:  291
Predator:  91
Empty:  9618
-----------------------------------------
Time Step:  92
Prey:  318
Predator:  90
Empty:  9592
-----------------------------------------
Time Step:  93
Prey:  323
Predator:  90
Empty:  9587
-----------------------------------------
Time Step:  94
Prey:  352
Predator:  93
Empty:  9555
-----------------------------------------
Time Step:  95
Prey:  354
Predator:  91
Empty:  9555
-----------------------------------------
Time Step:  96
Prey:  377
Predator:  95
Empty:  9528
-----------------------------------------
Time Step:  97
Prey:  374
Predator:  109
Empty:  9

Time Step:  172
Prey:  2305
Predator:  526
Empty:  7169
-----------------------------------------
Time Step:  173
Prey:  2354
Predator:  531
Empty:  7115
-----------------------------------------
Time Step:  174
Prey:  2440
Predator:  535
Empty:  7025
-----------------------------------------
Time Step:  175
Prey:  2430
Predator:  537
Empty:  7033
-----------------------------------------
Time Step:  176
Prey:  2440
Predator:  546
Empty:  7014
-----------------------------------------
Time Step:  177
Prey:  2480
Predator:  554
Empty:  6966
-----------------------------------------
Time Step:  178
Prey:  2526
Predator:  575
Empty:  6899
-----------------------------------------
Time Step:  179
Prey:  2539
Predator:  576
Empty:  6885
-----------------------------------------
Time Step:  180
Prey:  2594
Predator:  585
Empty:  6821
-----------------------------------------
Time Step:  181
Prey:  2626
Predator:  598
Empty:  6776
-----------------------------------------
Time Step:  182
Prey

Time Step:  256
Prey:  4871
Predator:  1342
Empty:  3787
-----------------------------------------
Time Step:  257
Prey:  4861
Predator:  1368
Empty:  3771
-----------------------------------------
Time Step:  258
Prey:  4909
Predator:  1396
Empty:  3695
-----------------------------------------
Time Step:  259
Prey:  4858
Predator:  1420
Empty:  3722
-----------------------------------------
Time Step:  260
Prey:  4876
Predator:  1415
Empty:  3709
-----------------------------------------
Time Step:  261
Prey:  4860
Predator:  1431
Empty:  3709
-----------------------------------------
Time Step:  262
Prey:  4886
Predator:  1457
Empty:  3657
-----------------------------------------
Time Step:  263
Prey:  4859
Predator:  1446
Empty:  3695
-----------------------------------------
Time Step:  264
Prey:  4899
Predator:  1473
Empty:  3628
-----------------------------------------
Time Step:  265
Prey:  4873
Predator:  1506
Empty:  3621
-----------------------------------------
Time Step:

Time Step:  339
Prey:  4588
Predator:  1958
Empty:  3454
-----------------------------------------
Time Step:  340
Prey:  4577
Predator:  1961
Empty:  3462
-----------------------------------------
Time Step:  341
Prey:  4494
Predator:  1965
Empty:  3541
-----------------------------------------
Time Step:  342
Prey:  4554
Predator:  1950
Empty:  3496
-----------------------------------------
Time Step:  343
Prey:  4536
Predator:  1940
Empty:  3524
-----------------------------------------
Time Step:  344
Prey:  4534
Predator:  1936
Empty:  3530
-----------------------------------------
Time Step:  345
Prey:  4472
Predator:  1960
Empty:  3568
-----------------------------------------
Time Step:  346
Prey:  4472
Predator:  1984
Empty:  3544
-----------------------------------------
Time Step:  347
Prey:  4421
Predator:  1986
Empty:  3593
-----------------------------------------
Time Step:  348
Prey:  4455
Predator:  1997
Empty:  3548
-----------------------------------------
Time Step:

...

Time Step:  840
Prey:  2744
Predator:  2696
Empty:  4560
-----------------------------------------
Time Step:  841
Prey:  2688
Predator:  2698
Empty:  4614
-----------------------------------------
Time Step:  842
Prey:  2692
Predator:  2675
Empty:  4633
-----------------------------------------
Time Step:  843
Prey:  2684
Predator:  2669
Empty:  4647
-----------------------------------------
Time Step:  844
Prey:  2686
Predator:  2650
Empty:  4664
-----------------------------------------
Time Step:  845
Prey:  2684
Predator:  2654
Empty:  4662
-----------------------------------------
Time Step:  846
Prey:  2706
Predator:  2685
Empty:  4609
-----------------------------------------
Time Step:  847
Prey:  2718
Predator:  2663
Empty:  4619
-----------------------------------------
Time Step:  848
Prey:  2712
Predator:  2622
Empty:  4666
-----------------------------------------
Time Step:  849
Prey:  2709
Predator:  2602
Empty:  4689
-----------------------------------------
Time Step:

Time Step:  923
Prey:  3029
Predator:  2675
Empty:  4296
-----------------------------------------
Time Step:  924
Prey:  3026
Predator:  2686
Empty:  4288
-----------------------------------------
Time Step:  925
Prey:  3053
Predator:  2698
Empty:  4249
-----------------------------------------
Time Step:  926
Prey:  3036
Predator:  2722
Empty:  4242
-----------------------------------------
Time Step:  927
Prey:  3054
Predator:  2704
Empty:  4242
-----------------------------------------
Time Step:  928
Prey:  3037
Predator:  2697
Empty:  4266
-----------------------------------------
Time Step:  929
Prey:  3089
Predator:  2702
Empty:  4209
-----------------------------------------
Time Step:  930
Prey:  3064
Predator:  2717
Empty:  4219
-----------------------------------------
Time Step:  931
Prey:  3055
Predator:  2686
Empty:  4259
-----------------------------------------
Time Step:  932
Prey:  3029
Predator:  2656
Empty:  4315
-----------------------------------------
Time Step:

Time Step:  1006
Prey:  2872
Predator:  2770
Empty:  4358
-----------------------------------------
Time Step:  1007
Prey:  2896
Predator:  2775
Empty:  4329
-----------------------------------------
Time Step:  1008
Prey:  2893
Predator:  2748
Empty:  4359
-----------------------------------------
Time Step:  1009
Prey:  2898
Predator:  2720
Empty:  4382
-----------------------------------------
Time Step:  1010
Prey:  2911
Predator:  2732
Empty:  4357
-----------------------------------------
Time Step:  1011
Prey:  2947
Predator:  2722
Empty:  4331
-----------------------------------------
Time Step:  1012
Prey:  2953
Predator:  2728
Empty:  4319
-----------------------------------------
Time Step:  1013
Prey:  2989
Predator:  2719
Empty:  4292
-----------------------------------------
Time Step:  1014
Prey:  3036
Predator:  2728
Empty:  4236
-----------------------------------------
Time Step:  1015
Prey:  3051
Predator:  2738
Empty:  4211
-----------------------------------------


Time Step:  1088
Prey:  2864
Predator:  2821
Empty:  4315
-----------------------------------------
Time Step:  1089
Prey:  2862
Predator:  2813
Empty:  4325
-----------------------------------------
Time Step:  1090
Prey:  2851
Predator:  2818
Empty:  4331
-----------------------------------------
Time Step:  1091
Prey:  2858
Predator:  2796
Empty:  4346
-----------------------------------------
Time Step:  1092
Prey:  2879
Predator:  2808
Empty:  4313
-----------------------------------------
Time Step:  1093
Prey:  2894
Predator:  2812
Empty:  4294
-----------------------------------------
Time Step:  1094
Prey:  2911
Predator:  2790
Empty:  4299
-----------------------------------------
Time Step:  1095
Prey:  2924
Predator:  2786
Empty:  4290
-----------------------------------------
Time Step:  1096
Prey:  2916
Predator:  2806
Empty:  4278
-----------------------------------------
Time Step:  1097
Prey:  2908
Predator:  2811
Empty:  4281
-----------------------------------------


KeyboardInterrupt: 